# Inference: Adaptive Covariance: localised MCMC

This example shows you how to perform Bayesian inference on a time series, using a variant of [Adaptive Covariance MCMC](http://pints.readthedocs.io/en/latest/mcmc_samplers/adaptive_covariance_mcmc_remi.html) detailed in Algorithm 7 of [1], which involves using a vector of possible proposal distributions.

[1] A tutorial on adaptive MCMC
    Christophe Andrieu and Johannes Thoms, Statistical Computing,
    2008, 18: 343-373

It follows on from the [first sampling example](./sampling-first-example.ipynb).

In [1]:
import os
os.chdir('C:/Users/bclamber/Desktop/AllPints/pints4/pints')
from __future__ import print_function
import pints
import pints.toy as toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt

# Load a forward model
model = toy.LogisticModel()

# Create some toy data
real_parameters = [0.015, 500]
times = np.linspace(0, 1000, 1000)
org_values = model.simulate(real_parameters, times)

# Add noise
noise = 10
values = org_values + np.random.normal(0, noise, org_values.shape)
real_parameters = np.array(real_parameters + [noise])

# Get properties of the noise sample
noise_sample_mean = np.mean(values - org_values)
noise_sample_std = np.std(values - org_values)

# Create an object with links to the model and time series
problem = pints.SingleOutputProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
log_likelihood = pints.UnknownNoiseLogLikelihood(problem)

# Create a uniform prior over both the parameters and the new noise variable
log_prior = pints.UniformLogPrior(
    [0.01, 400, noise*0.1],
    [0.02, 600, noise*100]
)

# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Choose starting points for 3 mcmc chains
xs = [
    real_parameters * 0.9,
]

# Create mcmc routine with four chains
mcmc = pints.MCMCSampling(log_posterior, 1, xs, method=pints.AdaptiveCovarianceLocalisedMCMC)

# Add stopping criterion
mcmc.set_max_iterations(4000)

# Start adapting after 1000 iterations
mcmc.set_initial_phase_iterations(1000)

# Disable logging mode
mcmc.set_log_to_screen(False)

# Run!
print('Running...')
chains = mcmc.run()
print('Done!')

# Show traces and histograms
pints.plot.trace(chains)

# Discard warm up
chains = chains[:, 2000:, :]

# Check convergence using rhat criterion
print('R-hat:')
print(pints.rhat_all_params(chains))

# Look at distribution in chain 0
pints.plot.pairwise(chains[0], kde=True)

# Show graphs
plt.show()

Running...
[  2.28980127e-02   5.48577795e+02   1.06025771e+01]
[  1.65000000e-02   5.50000000e+02   1.10000000e+01]
-1
[  1.14501289e-02   4.49493299e+02   8.87052700e+00]
[  1.35000000e-02   4.50000000e+02   9.00000000e+00]
-1
[  3.33430761e-02   5.74642416e+02   1.20963830e+01]
[  1.72500000e-02   5.75000000e+02   1.15000000e+01]
-1
[  1.01045202e-02   5.52498594e+02   1.09015421e+01]
[  1.65000000e-02   5.50000000e+02   1.10000000e+01]
-1
[  2.46059623e-02   4.51510855e+02   8.68970038e+00]
[  1.35000000e-02   4.50000000e+02   9.00000000e+00]
-1
[  1.24447753e-02   5.75589785e+02   1.18408322e+01]
[  1.72500000e-02   5.75000000e+02   1.15000000e+01]
-1
[  1.04284848e-02   5.51058665e+02   1.09227926e+01]
[  1.65000000e-02   5.50000000e+02   1.10000000e+01]
-1
[  2.11436947e-02   4.50184828e+02   8.80592217e+00]
[  1.35000000e-02   4.50000000e+02   9.00000000e+00]
-1
[ -1.52207616e-02   5.73929245e+02   1.15368505e+01]
[  1.24447753e-02   5.75589785e+02   1.18408322e+01]
-1
[  1.103

C:\Users\bclamber\Desktop\AllPints\pints4\pints\pints\_mcmc\_adaptive_covariance.py:162: RuntimeWarning: overflow encountered in exp
  self._alpha = np.minimum(1, np.exp(r))


[  2.20459308e-02   5.75354263e+02   1.22924079e+01]
[  1.34344442e-02   5.76662553e+02   1.22532312e+01]
-1
[  1.51221199e-02   5.48849401e+02   1.10548299e+01]
[  1.58208825e-02   5.49013553e+02   1.10073620e+01]
-1
[  1.04104086e-02   4.49057816e+02   9.00918328e+00]
[  1.62460927e-02   4.49201578e+02   9.10171643e+00]
-1
[  7.69438615e-03   5.75409400e+02   1.23753075e+01]
[  1.34344442e-02   5.76662553e+02   1.22532312e+01]
-1
[  1.69812549e-02   5.48404214e+02   1.10018453e+01]
[  1.51221199e-02   5.48849401e+02   1.10548299e+01]
-1
[  1.46917252e-02   4.48609620e+02   8.86843450e+00]
[  1.62460927e-02   4.49201578e+02   9.10171643e+00]
-1
[  1.35499081e-02   5.76468216e+02   1.22035210e+01]
[  1.34344442e-02   5.76662553e+02   1.22532312e+01]
-1
[  1.97414956e-02   5.49052178e+02   1.11837734e+01]
[  1.51221199e-02   5.48849401e+02   1.10548299e+01]
-1
[  1.48146142e-02   4.49181002e+02   8.96703102e+00]
[  1.62460927e-02   4.49201578e+02   9.10171643e+00]
-1
[  1.84234877e-02  

[  1.57247991e-02   4.49677564e+02   9.05181006e+00]
-1
[  1.44900513e-02   5.76903695e+02   1.22497587e+01]
[  1.40087622e-02   5.76932898e+02   1.22512340e+01]
-1
[  1.39524654e-02   5.45819234e+02   1.12160321e+01]
[  1.43008061e-02   5.45854712e+02   1.12257843e+01]
-1
[  1.62677571e-02   4.49629844e+02   9.04929974e+00]
[  1.57247991e-02   4.49677564e+02   9.05181006e+00]
-1
[  1.37855089e-02   5.76957472e+02   1.22633612e+01]
[  1.40087622e-02   5.76932898e+02   1.22512340e+01]
-1
[  1.36821151e-02   5.45723283e+02   1.12513611e+01]
[  1.43008061e-02   5.45854712e+02   1.12257843e+01]
-1
[  1.57538668e-02   4.49678374e+02   9.06777004e+00]
[  1.57247991e-02   4.49677564e+02   9.05181006e+00]
-1
[  1.39526304e-02   5.76949266e+02   1.22391923e+01]
[  1.40087622e-02   5.76932898e+02   1.22512340e+01]
-1
[  1.44161759e-02   5.45894843e+02   1.12213860e+01]
[  1.43008061e-02   5.45854712e+02   1.12257843e+01]
-1
[  1.56922054e-02   4.49532842e+02   9.07257214e+00]
[  1.57538668e-02  

[  1.43498462e-02   5.45718039e+02   1.12691445e+01]
[  1.43597535e-02   5.45714706e+02   1.12689283e+01]
-1
[  1.56006941e-02   4.49947693e+02   9.14014571e+00]
[  1.56829015e-02   4.49916441e+02   9.13835429e+00]
-1
[  1.37477169e-02   5.76603969e+02   1.22700680e+01]
[  1.38963208e-02   5.76577117e+02   1.22685178e+01]
-1
[  1.44621569e-02   5.45736119e+02   1.12668070e+01]
[  1.43498462e-02   5.45718039e+02   1.12691445e+01]
-1
[  1.59304603e-02   4.49941902e+02   9.13780839e+00]
[  1.56006941e-02   4.49947693e+02   9.14014571e+00]
-1
[  1.40921791e-02   5.76579371e+02   1.22636887e+01]
[  1.38963208e-02   5.76577117e+02   1.22685178e+01]
-1
[  1.43002605e-02   5.45700924e+02   1.12675923e+01]
[  1.43498462e-02   5.45718039e+02   1.12691445e+01]
-1
[  1.55105377e-02   4.49969126e+02   9.14219815e+00]
[  1.56006941e-02   4.49947693e+02   9.14014571e+00]
-1
[  1.37531044e-02   5.76567566e+02   1.22723119e+01]
[  1.38963208e-02   5.76577117e+02   1.22685178e+01]
-1
[  1.43713535e-02  

...

-1
[  1.43875749e-02   5.43385322e+02   1.13830066e+01]
[  1.43829372e-02   5.43391663e+02   1.13836179e+01]
-1
[  1.56285476e-02   4.54230984e+02   9.50453864e+00]
[  1.55947054e-02   4.54232156e+02   9.50529828e+00]
-1
[  1.39716344e-02   5.73607722e+02   1.26865836e+01]
[  1.39635754e-02   5.73609638e+02   1.26902973e+01]
-1
[  1.43855502e-02   5.43401617e+02   1.13815889e+01]
[  1.43875749e-02   5.43385322e+02   1.13830066e+01]
-1
[  1.56466242e-02   4.54253692e+02   9.50567824e+00]
[  1.55947054e-02   4.54232156e+02   9.50529828e+00]
-1
[  1.40060392e-02   5.73616284e+02   1.26849438e+01]
[  1.39635754e-02   5.73609638e+02   1.26902973e+01]
-1
[  1.43757571e-02   5.43361924e+02   1.13860290e+01]
[  1.43875749e-02   5.43385322e+02   1.13830066e+01]
-1
[  1.55836815e-02   4.54235145e+02   9.50529207e+00]
[  1.55947054e-02   4.54232156e+02   9.50529828e+00]
-1
[  1.39675951e-02   5.73603979e+02   1.26880720e+01]
[  1.39635754e-02   5.73609638e+02   1.26902973e+01]
-1
[  1.43598973e-0

-1
[  1.44280829e-02   5.42728013e+02   1.14158936e+01]
[  1.44162222e-02   5.42731331e+02   1.14144688e+01]
-1
[  1.55296080e-02   4.54669504e+02   9.56128677e+00]
[  1.55694908e-02   4.54653246e+02   9.55859674e+00]
-1
[  1.39407065e-02   5.73469951e+02   1.27736215e+01]
[  1.39666361e-02   5.73472665e+02   1.27744346e+01]
-1
[  1.44861469e-02   5.42715776e+02   1.14162177e+01]
[  1.44280829e-02   5.42728013e+02   1.14158936e+01]
-1
[  1.55283133e-02   4.54675682e+02   9.56091836e+00]
[  1.55296080e-02   4.54669504e+02   9.56128677e+00]
-1
[  1.39590650e-02   5.73475132e+02   1.27742772e+01]
[  1.39666361e-02   5.73472665e+02   1.27744346e+01]
-1
[  1.43899680e-02   5.42680682e+02   1.14169671e+01]
[  1.44280829e-02   5.42728013e+02   1.14158936e+01]
-1
[  1.54821916e-02   4.54652686e+02   9.55820216e+00]
[  1.55283133e-02   4.54675682e+02   9.56091836e+00]
-1
[  1.40177336e-02   5.73454347e+02   1.27806651e+01]
[  1.39666361e-02   5.73472665e+02   1.27744346e+01]
-1
[  1.43366895e-0

-1
[  1.56592030e-02   4.55524455e+02   9.67094858e+00]
[  1.56641727e-02   4.55493494e+02   9.66689011e+00]
-1
[  1.40489257e-02   5.73300660e+02   1.28371937e+01]
[  1.39882081e-02   5.73297270e+02   1.28380454e+01]
-1
[  1.44308220e-02   5.41986748e+02   1.14543825e+01]
[  1.43889288e-02   5.42054555e+02   1.14507862e+01]
-1
[  1.56451639e-02   4.55576573e+02   9.67838322e+00]
[  1.56592030e-02   4.55524455e+02   9.67094858e+00]
-1
[  1.39853839e-02   5.73293594e+02   1.28371761e+01]
[  1.39882081e-02   5.73297270e+02   1.28380454e+01]
-1
[  1.44456108e-02   5.42025971e+02   1.14520041e+01]
[  1.44308220e-02   5.41986748e+02   1.14543825e+01]
-1
[  1.56232944e-02   4.55567186e+02   9.67732494e+00]
[  1.56451639e-02   4.55576573e+02   9.67838322e+00]
-1
[  1.39690160e-02   5.73301815e+02   1.28344924e+01]
[  1.39882081e-02   5.73297270e+02   1.28380454e+01]
-1
[  1.43677621e-02   5.41963895e+02   1.14558001e+01]
[  1.44308220e-02   5.41986748e+02   1.14543825e+01]
-1
[  1.55932592e-0

[  1.40345736e-02   5.72768579e+02   1.30967470e+01]
[  1.39854668e-02   5.72778205e+02   1.30935345e+01]
-1
[  1.46217331e-02   5.36778650e+02   1.17291427e+01]
[  1.45734423e-02   5.37173604e+02   1.17068727e+01]
-1
[  1.55182743e-02   4.59654300e+02   1.01938427e+01]
[  1.55049473e-02   4.60082987e+02   1.02490176e+01]
-1
[  1.40592138e-02   5.72745990e+02   1.31080266e+01]
[  1.40345736e-02   5.72768579e+02   1.30967470e+01]
-1
[  1.46188914e-02   5.36943062e+02   1.17206667e+01]
[  1.46217331e-02   5.36778650e+02   1.17291427e+01]
-1
[  1.54901397e-02   4.60210665e+02   1.02656781e+01]
[  1.55049473e-02   4.60082987e+02   1.02490176e+01]
-1
[  1.40811013e-02   5.72766585e+02   1.31001010e+01]
[  1.40592138e-02   5.72745990e+02   1.31080266e+01]
-1
[  1.46468659e-02   5.36890112e+02   1.17226222e+01]
[  1.46217331e-02   5.36778650e+02   1.17291427e+01]
-1
[  1.54831585e-02   4.60094978e+02   1.02511601e+01]
[  1.54901397e-02   4.60210665e+02   1.02656781e+01]
-1
[  1.40800151e-02  

LinAlgError: singular matrix

In [ ]:
from sklearn import mixture
import numpy as np

In [ ]:
n_samples = 500
C = np.array([[0., -0.1], [1.7, .4]])
X = np.r_[np.dot(np.random.randn(n_samples, 2), C),
          .7 * np.random.randn(n_samples, 2) + np.array([-6, 3])]


In [ ]:
gmm = mixture.GaussianMixture(n_components=4, covariance_type='full').fit(X)

In [ ]:
gmm.aic(X)

In [ ]:
gmm.means_

In [ ]:
gmm.weights_

In [ ]:
[4 for i in range(0,5)]

In [ ]:
from scipy.stats import multivariate_normal
mu = [[1,1], [1,2]]
sigma = [[[1,0],[0,1]],[[2,0],[0,1]]]
i = 0
lambda x: multivariate_normal.logpdf(x, mu[i], self._sigma[i])

In [ ]:
from scipy.stats import multivariate_normal
lambda x: multivariate_normal.logpdf(x, mu[1], self._sigma[1])

In [ ]:
q_l = [lambda x: multivariate_normal.logpdf(x, mu[i], self._sigma[i]) for i in range(2)]

In [ ]:
np.sum([1,2,3])

In [ ]:
from scipy.misc import logsumexp

In [ ]:
logsumexp([1,2,3])

In [ ]:
np.array([1,2,3])-1

In [ ]:
np.random.choice(3, 1, [ 0.33333333,  0.33333333,  0.33333333])[0]

In [ ]:
np.random.choice(3, 1, np.array([ 0.33333333,  0.33333333,  0.33333333]))[0]

In [ ]:
a=np.array([ 0.33333333,  0.33333333,  0.33333333])
a.tolist()

In [ ]:
a

In [ ]:
np.exp(1)

In [ ]:
dsigm = np.reshape(np.array([1,2,3]) - np.array([0,0,0]), (3, 1))
np.all(np.linalg.eigvals(np.dot(dsigm, dsigm.T)))

In [ ]:
dsigm = np.reshape(np.array([ -1.31211664e-05,  1.15126055e-02,   1.75795761e-03]), (3,1))
np.linalg.eigvals(np.dot(dsigm, dsigm.T))

In [ ]:
np.linalg.det(np.array([[  2.31572494e-08,  -5.94207061e-04,   6.71774143e-05],
 [ -5.94207061e-04,   1.85538298e+01,  -2.09736990e+00],
 [  6.71774143e-05,  -2.09736990e+00,   2.37098737e-01]]))